## Scraper

### Make necessary requests

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import json
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
import re
import copy

### Setup initial variables

In [ ]:
baseURL = "https://www.linkedin.com/uas/login-submit"
redirectURL = "Place the profile URL"
userEmail = "Place your email here"
page404 = "https://www.linkedin.com/jsnfkljn/?_l=en_US"

### Get login page and csrf value

In [ ]:
webSession = requests.Session()
headers = {
"User-Agent":
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/110.0"
}
webSession.headers.update(headers)
loginPage = webSession.post(baseURL)
loginSoup = bs(loginPage.content, "html.parser")
# pprint(soup.prettify())

csrf = (loginSoup.find('input', attrs={'name': 'loginCsrfParam'}))['value']
pprint(csrf)

### Use the csrf and username and password to make an authenticated POST call

In [ ]:
payload_dict = {'session_key':userEmail,'session_password': 'Enter your password here','loginCsrfParam': csrf,'session_redirect': redirectURL}
loginSuccessPage = webSession.post(baseURL, data=payload_dict)

loginSuccessSoup = bs(loginSuccessPage.content, "html.parser")
pprint(payload_dict)
webSession.get(redirectURL)
# pprint(loginSuccessSoup.text)
pprint(webSession.cookies.get_dict())

# pprint(profileSoup.prettify())


### Set up Selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

# End existing sessions
try:
    driver.quit()
except:
    pass

# options = ChromeOptions()
options = FirefoxOptions()
options.add_argument('--lang=en')
# options.add_argument('--disable-blink-features=AutomationControlled')
# service = ChromeService(executable_path=ChromeDriverManager().install())
service = FirefoxService(executable_path=GeckoDriverManager().install())
# driver = webdriver.Chrome(service=service)
driver = webdriver.Firefox(service=service, options=options)
# driver.close()
# driver.implicitly_wait(10)

### Transfer session over to Selenium

In [ ]:
driver.get(page404)
authCookies = ['li_at', 'liap', 'leadgen.api_session', 'transaction_state', 'lihc_auth_str',\
    'lihc_auth_', 'li_a', 'li_ep_auth_context', 'sj_sessionid', 'cp_session', 'G_AUTHUSER_H',\
        'G_ENABLED_IDPS', 'g_state', 'x-ms-cpim-cache|<id>', 'x-ms-cpim-trans',\
        'x-ms-cpim-sso:linkedinenterprise.onmicrosoft.com_0', 'lidc', 'li_rm']

reqCookies = webSession.cookies.get_dict()
# pprint(reqCookies)
for cookie in reqCookies:
    # if(cookie not in authCookies): continue
    # pprint({
    #     'name': cookie, 
    #     'value': reqCookies[cookie],
    # })
    driver.add_cookie({
        'name': cookie, 
        'value': reqCookies[cookie],
    })

## Copy paste cookies from browser
    
# pprint(driver.get_cookies())
# WebDriverWait(driver,5).until(EC.presence_of_element_located(By.XPATH('//button[text()="Some text"]')))
WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(),'Page not found')]")))

### Scape the profile!

#### Load the profile into the driver

In [ ]:
def document_initialised(driver):
    soup = bs(driver.page_source, 'html.parser')
    # pprint(soup.prettify())
    try:
        profileSection = soup.find('div', {'class':'mt2 relative'})
        aboutSection = soup.find('div', {'id':'about'})
        # pprint(profileSection)
        if(profileSection==None or aboutSection==None):
            raise Exception('None')
        # profileSection = profileSection['pv-text-details__left-panel']
    
        return True
    except:
        return False


driver.get("https://www.linkedin.com/in/palathingal/")
try:
    WebDriverWait(driver, timeout=5).until(document_initialised)
except:
    pass
# driver.switch_to.new_window('tab');
# try:
# except:
#     pass
    # driver.get(redirectURL)
profileSoup = bs(driver.page_source, "html.parser")

In [ ]:
profileScraped={
    'name':"",
    'titleDescription':"",
    'location': "",
    'about': "",
    'experiences': "",
    'education': "",
    'recommendations':"",
}

#### Scrape the Name

In [ ]:
# Go to the fist ember (always the profile)
# Go down the path for the profile name
try:
    res = profileSoup.find('section', {'id': re.compile('ember.*')})\
        .find('div', {'class':'ph5'})\
        .find('div', {'class':'mt2 relative'})\
        .div\
        .div\
        .h1\
        .text
except:
    res = 'N/A'

print(res)
profileScraped['name'] = res


##### Scrape the Location

In [ ]:
try:
    res = profileSoup.find('section', {'id': re.compile('ember.*')})\
        .find('div', {'class':'ph5'})\
        .find('div', {'class':'mt2 relative'})\
        .find_all('div', recursive=False)[1]\
        .span\
        .text
except:
    res = 'N/A'

print(res)
profileScraped['location'] = res


#### Scrape the Title's Description

In [ ]:
# Go to the fist ember (always the profile)
# Go down the path for the profile name
try:
    res = profileSoup.find('section', {'id': re.compile('ember.*')})\
        .find('div', {'class':'ph5'})\
        .find('div', {'class':'mt2 relative'})\
        .div\
        .find_all('div', recursive=False)[1]\
        .text\
        .strip()
except:
    res = 'N/A'

print(res)
profileScraped['titleDescription'] = res


#### Scrape the About section

In [ ]:
# Go to the ember with a child with id='about'
# Go down the path for the description
def aboutFilter(elem):
    # if(elem.find('div',{'id': 'about'})): pprint(elem)
    try:
        return elem.name=='section' and elem.find('div',{'id': 'about'})
    except:
        return False

try:
    res = profileSoup.find(aboutFilter)\
        .find('div', {'class':'ph5'})\
        .div\
        .div\
        .div\
        .span\
        .text
except:
    res = 'N/A'

print(res)
profileScraped['about'] = res


#### Scrape Experience

##### Navigate to the 'Experience' page

In [ ]:
# def experience_initialised(driver):
#     driver.
driver.get(redirectURL+'details/experience/')
try:
    WebDriverWait(driver, timeout=5).until(lambda d: d.find_element(By.TAG_NAME,"ul") and d.find_element(By.CLASS_NAME,"pvs-list"))
except:
    pass
expSoup = bs(driver.page_source, 'html.parser')
expSoup = expSoup.find('ul', {'class':'pvs-list'})

##### Finally Scrape Experience!

In [ ]:
experience = []
pastExperience = []
try:
    pastExperience = expSoup.find_all('li', recursive=False)
except:
    pass

for exp in pastExperience:
    job = {}
    subjobs=[]
    try:
        subjobs = exp.div.div.find_all('div', recursive=False)[1].find_all('div', recursive=False)[1].ul.li.div.div.div.ul.find_all('li', recursive=False)
    except:
        pass
    # pprint(subjobs[0])
    if(len(subjobs)!=0):
        for sj in subjobs:
            job={}
            try:
                job['company'] = exp.div\
                            .div\
                            .find_all('div', recursive=False)[1]\
                            .div\
                            .a\
                            .div\
                            .span\
                            .span\
                            .text
            except:
                job['company'] = 'N/A'
            
            try:
                job['location'] = exp.div\
                            .div\
                            .find_all('div', recursive=False)[1]\
                            .div\
                            .a\
                            .find('span', recursive=False, attrs={'class': 't-black--light'})\
                            .span\
                            .text
            except:
                try:
                    job['location'] = sj.div\
                        .div\
                        .find_all('div', recursive=False)[1]\
                        .div\
                        .a\
                        .find_all('span', recursive=False, attrs={'class': 't-black--light'})[1]\
                        .span\
                        .text
                except:
                    job['location'] = 'N/A'
            
            try:
                job['position'] = sj.div\
                    .div\
                    .find_all('div', recursive=False)[1]\
                    .div\
                    .a\
                    .div\
                    .span\
                    .span\
                    .text
            except:
                job['position'] = 'N/A'
            
            try:
                job['dateRange'] = sj.div\
                    .div\
                    .find_all('div', recursive=False)[1]\
                    .div\
                    .a\
                    .find('span', recursive=False, attrs={'class': 't-black--light'})\
                    .span\
                    .text
            except:
                job['dateRange'] = 'N/A'
            
            try:
                job['description'] = sj.div\
                    .div\
                    .find_all('div', recursive=False)[1]\
                    .find_all('div', recursive=False)[1]\
                    .ul\
                    .li\
                    .text\
                    .strip()
            except:
                job['description'] = 'N/A'
                
            experience+=[copy.deepcopy(job)]

        continue
            
        
    try:
        job['position'] = exp.div\
            .div\
            .find_all('div', recursive=False)[1]\
            .div\
            .div\
            .div\
            .span\
            .span\
            .text
    except:
        job['position'] = 'N/A'
    try:
        job['company'] = exp.div\
            .div\
            .find_all('div', recursive=False)[1]\
            .div\
            .div\
            .find('span', recursive=False)\
            .span\
            .text\
            .split('·')[0]
    except:
        job['company'] = 'N/A'
    
    try:
        job['job'] = exp.div\
            .div\
            .find_all('div', recursive=False)[1]\
            .div\
            .div\
            .find('span', recursive=False)\
            .span\
            .text\
            .split('·')[1]
    except:
        job['job'] = 'N/A'
    
    try:
        job['dateRange'] = exp.div\
            .div\
            .find_all('div', recursive=False)[1]\
            .div\
            .div\
            .find_all('span', recursive=False)[1]\
            .span\
            .text
    except:
        job['dateRange'] = 'N/A'
            
    try:
        job['location'] = exp.div\
            .div\
            .find_all('div', recursive=False)[1]\
            .div\
            .div\
            .find_all('span', recursive=False)[2]\
            .span\
            .text
    except:
        job['location'] = 'N/A'

    try:
        job['description'] = exp.div\
            .div\
            .find_all('div', recursive=False)[1]\
            .find_all('div', recursive=False)[1]\
            .ul\
            .li\
            .div\
            .ul\
            .li\
            .div\
            .div\
            .div\
            .span\
            .text
    except:
        job['description'] = 'N/A'
            
    experience+=[job]
pprint(experience)
profileScraped['experience'] = experience

#### Scrape Education

##### Navigate to the 'Education' page

In [ ]:
driver.get(redirectURL+'details/education/')
try:
    WebDriverWait(driver, timeout=7).until(lambda d: d.find_element(By.TAG_NAME,"ul") and d.find_element(By.CLASS_NAME,"pvs-list") and d.find_element(By.CLASS_NAME,"ivm-image-view-model"))
except:
    pass
edSoup = bs(driver.page_source, 'html.parser')
edSoup = edSoup.find('ul', {'class':'pvs-list'})

##### Finally Scrape Education!

In [ ]:
education = []
pastEducation = []
try:
    pastEducation = edSoup.find_all('li', recursive=False)
except:
    pass

for current in pastEducation:
    school = {}
    try:
        school['name'] = current.div\
            .div\
            .find_all('div', recursive=False)[1]\
            .div\
            .a\
            .div\
            .span\
            .span\
            .text
    except:
        school['name'] = 'N/A'
    try:
        school['degree'] = current.div\
            .div\
            .find_all('div', recursive=False)[1]\
            .div\
            .a\
            .find('span',class_="t-14 t-normal", recursive=False)\
            .span\
            .text
    except:
        school['degree'] = "N/A"

    try:
        school['dateRange'] = current.div\
            .div\
            .find_all('div', recursive=False)[1]\
            .div\
            .a\
            .find('span', {'class': 't-black--light'}, recursive=False)\
            .span\
            .text
        
    except:
        school['dateRange'] = 'N/A'
        
    try:
        school['description'] = current.div\
            .div\
            .find_all('div', recursive=False)[1]\
            .find_all('div', recursive=False)[1]\
            .text\
        
        school['description'] = (re.sub(r'\n\s*\n', '\n\n', school['description'])).strip()
        
    except:
        school['description'] = 'N/A'
    
    education+=[school]

print(education)
profileScraped['education'] = education

#### Scrape Recommendations

##### Navigate to Recommendations page

In [ ]:
driver.get(redirectURL+'details/recommendations/')
WebDriverWait(driver, timeout=7).until(lambda d: d.find_element(By.TAG_NAME,"ul") and d.find_element(By.CLASS_NAME,"pvs-list") and d.find_element(By.CLASS_NAME,"t-black--light"))
recSoup = bs(driver.page_source, 'html.parser')
recSoup = recSoup.find('ul', {'class':'pvs-list'})

##### Finally Scrape Recommendations!

In [ ]:
recommendations = []
pastRecommendations = recSoup.find_all('li', recursive=False)

for rec in pastRecommendations:
    recLi = {}
    try:
        recLi['name'] = rec.div\
            .div\
            .find_all('div', recursive=False)[1]\
            .div\
            .a\
            .div\
            .span\
            .span\
            .text
    except:
        recLi['name'] = 'N/A'
    
    try:
        recLi['profile'] = rec.div\
            .div\
            .find_all('div', recursive=False)[1]\
            .div\
            .a\
            .get('href')
    except:
        recLi['profile'] = 'N/A'

    try:
        subtitle = rec.div\
            .div\
            .find_all('div', recursive=False)[1]\
            .div\
            .a\
            .find('span', attrs = {'class': 't-black--light'}, recursive=False)\
            .span\
            .text\
            .split()
        
        recLi['date'] = ' '.join(subtitle[:3])
        recLi['relationship'] = ' '.join(subtitle[3:])
    except Exception as e:
        print(e)
        recLi['date'] = 'N/A'
        recLi['relationship'] = 'N/A'
    
    try:
        recLi['testimonial'] = rec.div\
            .div\
            .find_all('div', recursive=False)[1]\
            .find_all('div', recursive=False)[1]\
            .find('div', attrs={'class': 't-normal'})\
            .span\
            .text
    except:
        recLi['testimonial'] = 'N/A'
    
    recommendations+=[recLi]

pprint(recommendations)
profileScraped['recommendations'] = recommendations

### For testing

In [ ]:
from linkedin_scraper import Person
person = Person("Profile URL", driver=driver)

## Testing Voyager API

In [ ]:
baseURL = "https://www.linkedin.com/voyager/api"
authURL = "https://www.linkedin.com/uas/authenticate"

In [ ]:
from  linkedin_api import Linkedin
api = Linkedin('LinkedIn Email', 'LinkedIn Password')

In [ ]:
profile = api.get_profile('Profile Tag (what comes after linkedin.com/in/....)')

In [ ]:
pprint(profile)

In [ ]:
newprof = {
    'name': ' '.join((profile.get('firstName', ''), profile.get('lastName', ''))),
    'titleDescription': profile.get('headline', 'N/A'),
    'location': ', '.join((profile.get('geoLocationName', ''), profile.get('geoCountryName', ''))),
    'about': profile.get('summary', 'N/A'),
}
newprof['experiences'] = []
for experience in profile['experience']:
    newExp = {
        'position': experience.get('title', 'N/A'),
        'company': experience.get('companyName', 'N/A'),
        'dateRange': str(experience.get('timePeriod', {}).get('startDate', {}).get('year', 'N/A'))+' - '+ str(experience.get('timePeriod', {}).get('endDate', {}).get('year', 'Present')),
        'location': experience.get('geoLocationName', 'N/A'),
        'description': experience.get('description', 'N/A'),
    }
    newprof['experiences']+=[newExp]
newprof['education'] = []
for ed in profile['education']:
    newEd = {
        'school': ed.get('school', {}).get('schoolName', 'N/A'),
        'degree': ed.get('degreeName', 'N/A'),
        'dateRange': str(ed.get('timePeriod', {}).get('startDate', {}).get('year', 'N/A'))+' - '+ str(ed.get('timePeriod', {}).get('endDate', {}).get('year', 'Present')),
        'description': '\n'.join((ed.get('activities', ''), ed.get('description', ''))).strip(),
    }
    newprof['education']+=[newExp]

pprint(newprof)